In [1]:
import os
import re
import numpy as np
import pandas as pd
import shutil
import string
import tensorflow as tf
import tensorflow_datasets as tfds
import tqdm

from tensorflow.keras import Model, preprocessing
from tensorflow.keras.layers import Dot, Embedding , Flatten
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [2]:
%load_ext tensorboard 

In [3]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [4]:
dataset , info = tfds.load('yelp_polarity_reviews', with_info= True , as_supervised=True)
train_dataset , test_dataset = dataset['train'] , dataset['test']
train_dataset.element_spec

Shuffling and writing examples to /root/tensorflow_datasets/yelp_polarity_reviews/plain_text/0.1.0.incompleteG5NH39/yelp_polarity_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/yelp_polarity_reviews/plain_text/0.1.0.incompleteG5NH39/yelp_polarity_reviews-test.tfrecord


Dataset yelp_polarity_reviews downloaded and prepared to /root/tensorflow_datasets/yelp_polarity_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [5]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [6]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [7]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [8]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

In [9]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [10]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

Epoch 1/10
8750/8750 [==============================] - 1021s 114ms/step - loss: 0.2564 - accuracy: 0.8861 - val_loss: 0.2127 - val_accuracy: 0.9130
Epoch 2/10
8750/8750 [==============================] - 1012s 116ms/step - loss: 0.2215 - accuracy: 0.9057 - val_loss: 0.1908 - val_accuracy: 0.9130
Epoch 3/10
8750/8750 [==============================] - 1003s 115ms/step - loss: 0.2068 - accuracy: 0.9116 - val_loss: 0.1738 - val_accuracy: 0.9208
Epoch 4/10
8750/8750 [==============================] - 1010s 115ms/step - loss: 0.1874 - accuracy: 0.9207 - val_loss: 0.1604 - val_accuracy: 0.9401
Epoch 5/10
8750/8750 [==============================] - 997s 114ms/step - loss: 0.1725 - accuracy: 0.9277 - val_loss: 0.1438 - val_accuracy: 0.9469
Epoch 6/10
8750/8750 [==============================] - 1001s 114ms/step - loss: 0.1623 - accuracy: 0.9325 - val_loss: 0.1411 - val_accuracy: 0.9464
Epoch 7/10
8750/8750 [==============================] - 1010s 115ms/step - loss: 0.1553 - accuracy: 0.9354 

In [11]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

594/594 [==============================] - 30s 51ms/step - loss: 0.1464 - accuracy: 0.9423
Test Loss: 0.14642906188964844
Test Accuracy: 0.9422894716262817


In [12]:
!mkdir -p saved_model
model.save('saved_model/SA_model')

INFO:tensorflow:Assets written to: saved_model/SA_model/assets


INFO:tensorflow:Assets written to: saved_model/SA_model/assets
